In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import os
#s.path.exists("/content/drive/MyDrive/GNN/10mm/result")

True

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import tensorflow.compat.v1 as tf
import random
import pandas as pd
import torch_scatter
import torch.nn as nn
from torch.nn import Linear, Sequential, LayerNorm, ReLU
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.data import DataLoader

import numpy as np
import time
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy
import matplotlib.pyplot as plt
import os

import tensorflow.compat.v1 as tf
import functools
import json
from torch_geometric.data import Data
import enum
import itertools

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def triangles_to_edges(faces):
  """Computes mesh edges from triangles.
     Note that this triangles_to_edges method was provided as part of the
     code release for the MeshGraphNets paper by DeepMind, available here:
     https://github.com/deepmind/deepmind-research/tree/master/meshgraphnets
  """
  # collect edges from triangles
  edges = tf.concat([faces[:, 0:2],
                     faces[:, 1:3],
                     tf.stack([faces[:, 2], faces[:, 0]], axis=1)], axis=0)
  # those edges are sometimes duplicated (within the mesh) and sometimes
  # single (at the mesh boundary).
  # sort & pack edges as single tf.int64
  receivers = tf.reduce_min(edges, axis=1)
  senders = tf.reduce_max(edges, axis=1)
  packed_edges = tf.bitcast(tf.stack([senders, receivers], axis=1), tf.int64)
  # remove duplicates and unpack
  unique_edges = tf.bitcast(tf.unique(packed_edges)[0], tf.int32)
  senders, receivers = tf.unstack(unique_edges, axis=1)
  # create two-way connectivity
  return (tf.concat([senders, receivers], axis=0),
          tf.concat([receivers, senders], axis=0))

from pandas._libs import internals
class NodeType(enum.IntEnum):

    internal = 0
    upper_lower = 1
    SIZE = 2

In [ ]:
data_file="/Data/csv/"

file_list=os.listdir(data_file)
print(len(file_list))
file_list

744


['0704_KJM_L_10_nodes.csv',
 '0704_KJM_L_10_surfnodes.csv',
 '0704_KJM_L_10_elements.csv',
 '0704_KJM_L_10_surfelements.csv',
 '0704_OYJ_L_10_nodes.csv',
 '0704_OYJ_L_10_surfnodes.csv',
 '0704_OYJ_L_10_elements.csv',
 '0704_OYJ_L_10_surfelements.csv',
 '0708_BJH_R_10_nodes.csv',
 '0708_BJH_R_10_surfnodes.csv',
 '0708_BJH_R_10_elements.csv',
 '0708_BJH_R_10_surfelements.csv',
 '0704_LGC_L_10_nodes.csv',
 '0704_LGC_L_10_surfnodes.csv',
 '0704_LGC_L_10_elements.csv',
 '0704_LGC_L_10_surfelements.csv',
 '0704_MSG_R_10_nodes.csv',
 '0704_MSG_R_10_surfnodes.csv',
 '0704_MSG_R_10_elements.csv',
 '0704_MSG_R_10_surfelements.csv',
 '0708_BJH_L_10_nodes.csv',
 '0708_BJH_L_10_surfnodes.csv',
 '0708_BJH_L_10_elements.csv',
 '0708_BJH_L_10_surfelements.csv',
 '0708_JHY_L_10_nodes.csv',
 '0708_JHY_L_10_surfnodes.csv',
 '0708_JHY_L_10_elements.csv',
 '0708_JHY_L_10_surfelements.csv',
 '0704_LGC_R_10_nodes.csv',
 '0704_LGC_R_10_surfnodes.csv',
 '0704_LGC_R_10_elements.csv',
 '0704_LGC_R_10_surfelement

In [ ]:
def create_dataset3(data_file):
    data_list=[]
    file_name_list=[]
    file_list=os.listdir(data_file)
    j=1
    for item in file_list:

        design_name=item.rpartition('_')[0]
        print(j)
        if design_name not in file_name_list and design_name != ".DS" :

            info=design_name.split("_")
            #extra_features=torch.tensor([float(info[-5])*1e-9, float(info[-3])*1e-3, float(info[-1])*np.pi/180])
            #extra_features=torch.tensor([float(info[-5])*1e-11, float(info[-3])*1e-5])

            file_name_list.append(design_name)

            design_node=data_file+'/'+design_name+'_nodes.csv'#id + coordicate + type
            design_element=data_file+'/'+design_name+'_elements.csv'#id + 4 nodes id

            df = pd.read_csv(design_node)
            df_node_indices_four=pd.read_csv(design_element)

            df_node_indices_four = df_node_indices_four.drop('Unnamed: 0', axis=1)
            df = df.drop('Unnamed: 0', axis=1)

            # Convert a four-node 3D element into three-node faces.

            df_node_indices_three = pd.DataFrame( columns = ['elem1','elem2','elem3'])
            df_node_indices_three2 = pd.DataFrame( columns = ['elem1','elem2','elem3'])


            for i in range(df_node_indices_four.shape[0]):
                four_nodes=df_node_indices_four[i:i+1].values.tolist()[0][-4:]

                three_nodes=np.array(list(itertools.combinations(four_nodes,3)))
                cell_df=pd.DataFrame(three_nodes, columns = ['elem1','elem2','elem3'])
                df_node_indices_three = df_node_indices_three.append(cell_df)

                three_nodes2=np.array(list(itertools.permutations(four_nodes,3)))
                cell_df2=pd.DataFrame(three_nodes2, columns = ['elem1','elem2','elem3'])
                df_node_indices_three2 = df_node_indices_three2.append(cell_df2)

            ########################### sun ###########################
            #The task of organizing data by retaining only the elements corresponding to the given node ID and removing all other elements.
            node_id = df['node_id'].to_list()

            df_node_indices_three = df_node_indices_three.astype(int)
            df_node_indices_three = df_node_indices_three[df_node_indices_three['elem1'].isin(node_id)]
            df_node_indices_three = df_node_indices_three[df_node_indices_three['elem2'].isin(node_id)]
            df_node_indices_three = df_node_indices_three[df_node_indices_three['elem3'].isin(node_id)]


            df_node_indices_three2 = df_node_indices_three2.astype(int)
            df_node_indices_three2 = df_node_indices_three2[df_node_indices_three2['elem1'].isin(node_id)]
            df_node_indices_three2 = df_node_indices_three2[df_node_indices_three2['elem2'].isin(node_id)]
            df_node_indices_three2 = df_node_indices_three2[df_node_indices_three2['elem3'].isin(node_id)]

            ########################### sun ###########################


            cells_index2=np.vstack((df_node_indices_three2["elem1"].to_numpy(),df_node_indices_three2["elem2"].to_numpy(),df_node_indices_three2["elem3"].to_numpy())).T.astype('int32')
            cells2=torch.tensor(cells_index2)


            # cells (element indices)
            cells_index=np.vstack((df_node_indices_three["elem1"].to_numpy(),df_node_indices_three["elem2"].to_numpy(),df_node_indices_three["elem3"].to_numpy())).T.astype('int32')
            cells=torch.tensor(cells_index)

            # node positions
            node_position=np.vstack((df["x"].to_numpy(),df["y"].to_numpy(),df["z"].to_numpy())).T.astype('float32')
            mesh_pos=torch.tensor(node_position)

            # disp, strain, stress information
            # stress=torch.tensor(np.vstack((df["S11"].to_numpy(),df["S22"].to_numpy(),df["S33"].to_numpy(),df["S12"].to_numpy(),df["S13"].to_numpy(),df["S12.1"].to_numpy())).T.astype('float32'))
            # stress11=torch.unsqueeze(torch.tensor(df["S11"].to_numpy().T.astype('float32')), 1)
            # stress22=torch.unsqueeze(torch.tensor(df["S22"].to_numpy().T.astype('float32')), 1)
            # stress33=torch.unsqueeze(torch.tensor(df["S33"].to_numpy().T.astype('float32')), 1)
            # stress12=torch.unsqueeze(torch.tensor(df["S12"].to_numpy().T.astype('float32')), 1)
            # stress13=torch.unsqueeze(torch.tensor(df["S13"].to_numpy().T.astype('float32')), 1)
            # stress23=torch.unsqueeze(torch.tensor(df["S12.1"].to_numpy().T.astype('float32')), 1)


            U=torch.tensor(np.vstack((df["U1"].to_numpy(),df["U2"].to_numpy(),df["U3"].to_numpy())).T.astype('float32'))

            # get edge indices
            edges = triangles_to_edges(tf.convert_to_tensor(cells_index))
            edge_index = torch.cat( (torch.tensor(edges[0].numpy()).unsqueeze(0) ,
                         torch.tensor(edges[1].numpy()).unsqueeze(0)), dim=0).type(torch.long)

            # get edge features
            u_i=torch.tensor(node_position)[edge_index[0]]
            u_j=torch.tensor(node_position)[edge_index[1]]
            u_ij=u_i-u_j #엣지 벡터 계산
            u_ij_norm = torch.norm(u_ij,p=2,dim=1,keepdim=True)
            edge_attr = torch.cat((u_ij,u_ij_norm),dim=-1).type(torch.float)

            # get input x
            #node_type_info=np.array([[df[item:item+1]["nodetype"].item()] for item in range(len(df))], dtype=np.int32)
            node_type_info=df["node_type"].to_numpy().T.astype('int32')
            node_type = torch.tensor(np.array(tf.one_hot(tf.convert_to_tensor(node_type_info), NodeType.SIZE))).squeeze(1)
            x_without_features = torch.cat((mesh_pos,node_type),dim=-1).type(torch.float)

            #extra_features_stack=extra_features.repeat(x_without_features.shape[0],1)
            #x_=torch.cat((mesh_pos,extra_features_stack),dim=-1).type(torch.float)
            #x=torch.cat((x_,node_type),dim=-1).type(torch.float)
            #x = x_without_features

            # get data ist
            data_list.append(Data(x=x_without_features, x_without_features=x_without_features ,
                                  edge_index=edge_index, edge_attr=edge_attr, deform=U,
                                  cells=cells, cells2=cells2,mesh_pos=mesh_pos, name=design_name ))
        else:
            pass
        j=j+1
    print("Done collecting data!")


    return torch.save(data_list, os.path.join("Data/pt_input" + '/0919_sample_final.pt'))




In [ ]:
create_dataset3(data_file)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
